In [20]:
#from lxml import html
#import requests
#import string
#import BeautifulSoup
#import pandas as pd
#import urlopen

from urllib import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string
from lxml import html

filename = r'file:///Users\Melissa\Documents\Udacity\Untitled Folder\High School Economics.htm'
file = urlopen(filename)

soup = BeautifulSoup(file, "lxml")
standards = soup.ul

subject = 'Economics'
standards_list = []
keys = ['Subject','Parent Text', 'Parent TID', 'Child Text', 'Child TID']

In [11]:
#print standards

In [12]:
first_parent_text = standards.li.a.contents
print first_parent_text

[u'Economic Components']


In [13]:
first_parent = standards.li
#print first_parent

In [14]:
#second_parent = first_parent.next_sibling
#second_parent_text = first_parent.a.contents
#print second_parent_text
#print second_parent

In [21]:
#second_parent_children = second_parent.ul.findAll("a")
#print second_parent_children

parent = standards.li

while parent != None:
    parent_text = parent.a.contents
    parent_tid = parent.a['href'][-7:]
    #print parent
    children = parent.ul.findAll("a")
    #print children
    #loop through children
    for child in children:
        child_text = child.contents
        child_tid = child['href'][-7:]
        standards_list.append(dict(zip(keys, [subject, parent_text, parent_tid, child_text, child_tid])))    
    parent = parent.next_sibling
print standards_list

[{'Parent TID': '3938544', 'Child TID': '4128511', 'Child Text': [u'Agriculture'], 'Parent Text': [u'Economic Components'], 'Subject': 'Economics'}, {'Parent TID': '3938544', 'Child TID': '2703845', 'Child Text': [u'Economic Institutions'], 'Parent Text': [u'Economic Components'], 'Subject': 'Economics'}, {'Parent TID': '3938544', 'Child TID': '2703827', 'Child Text': [u'Economic Systems'], 'Parent Text': [u'Economic Components'], 'Subject': 'Economics'}, {'Parent TID': '3938544', 'Child TID': '2703828', 'Child Text': [u'Factors of Production'], 'Parent Text': [u'Economic Components'], 'Subject': 'Economics'}, {'Parent TID': '3938544', 'Child TID': '2703844', 'Child Text': [u'Financial Institutions'], 'Parent Text': [u'Economic Components'], 'Subject': 'Economics'}, {'Parent TID': '3938544', 'Child TID': '2703826', 'Child Text': [u'Free Market'], 'Parent Text': [u'Economic Components'], 'Subject': 'Economics'}, {'Parent TID': '3938544', 'Child TID': '4128512', 'Child Text': [u'Labor'],

In [23]:
standards_list_df = pd.DataFrame(standards_list)
print standards_list_df

   Child TID                               Child Text Parent TID  \
0    4128511                            [Agriculture]    3938544   
1    2703845                  [Economic Institutions]    3938544   
2    2703827                       [Economic Systems]    3938544   
3    2703828                  [Factors of Production]    3938544   
4    2703844                 [Financial Institutions]    3938544   
5    2703826                            [Free Market]    3938544   
6    4128512                                  [Labor]    3938544   
7    2703843                     [Medium of Exchange]    3938544   
8    2703841                                  [Taxes]    3938544   
9    2703835                             [Technology]    3938544   
10   2703846                         [Business Cycle]    3938542   
11   2703830                                 [Choice]    3938542   
12   4128513                            [Competition]    3938542   
13   2703823                   [Economic Develop

In [25]:
#output to Excel
output = subject + "Export.xlsx"

writer = pd.ExcelWriter(output)
standards_list_df.to_excel(writer, sheet_name='Sheet1')
writer.save()